In [1]:
import subprocess
import os
import glob
import pandas as pd
import json
cwd = os.getcwd()
cwd_exe =os.path.join(cwd,'softBV0405.exe')
all_files = glob.glob(os.path.join(cwd[:-4],'test/Li_database_CN4_occ1/*.cif'))

In [217]:
# Load Unitary Database
uni_df = UnitaryDF()

# Main process to fill cifDic
cifDic = {}
for cif in all_files[:2]:
    cif_name = cif[68:]
    cifDic[cif_name] = {}
    process = subprocess.run([cwd_exe, "--print-cell", cif], cwd=r'd:\Study\softBV_mix\GitHub\projects\Coord\bin', capture_output=True)
    stdout = str(process.stdout, "utf-8")
    Sites, name = fillSitesFromStdout(stdout)
    SiteDic = fillSiteDic(Sites,cwd_exe,cif)
    SiteDic = calAngleToSiteDic(SiteDic)
    SiteDic = fillUniInfo(SiteDic, uni_df)
    cifDic[cif_name]['Sites'] = SiteDic
    cifDic[cif_name]['Name'] = name
    
# Load cifDic into AngleList
AngleList = [[]]
i=0
for cif in cifDic:
    if len(cifDic[cif]) < 1:
        print(cif)
        continue
    for site in cifDic[cif]['Sites']:
        # Append cif name
        AngleList[i].append(cif)
        # Append chemial name
        AngleList[i].append(cifDic[cif]['Name'])
        # Append Site
        AngleList[i].append(site)
        # Append OS
        AngleList[i].append((cifDic[cif]['Sites'][site]['OS']))
        # Append Period
        AngleList[i].append((cifDic[cif]['Sites'][site]['Period']))
        # Append Group
        AngleList[i].append((cifDic[cif]['Sites'][site]['Group']))
        # Append CN
        AngleList[i].append(float(cifDic[cif]['Sites'][site]['CN']))
        # Append Angle list
        AngleList[i].append(cifDic[cif]['Sites'][site]['Angles'])
        # Add next column
        AngleList.append([])
        i += 1

In [207]:
# Fill unitary information to SiteDic
def fillUniInfo(SiteDic, uni_df):
    for site in SiteDic:
        period = uni_df[(uni_df['ty']== SiteDic[site]['Element']) & (uni_df['os']==int(SiteDic[site]['OS']))]['n'].tolist()
        if len(period) == 1:
            SiteDic[site]['Period'] = period[0]
        else:
            print("Element period is not found for",site,"!\n")
        group = uni_df[(uni_df['ty']== SiteDic[site]['Element']) & (uni_df['os']==int(SiteDic[site]['OS']))]['g'].tolist()
        if len(group) == 1:
            SiteDic[site]['Group'] = group[0]
        else:
            print("Element Group is not found for",site,"!\n")
    return SiteDic

In [9]:

process = subprocess.run([cwd_exe, "--print-cell", '../test/Combine/100465.cif'], cwd=r'd:\Study\softBV_mix\GitHub\projects\Coord\bin', capture_output=True)
stdout = str(process.stdout, "utf-8")
''''
stdout_lines = stdout.split("\r\n")
Sites = {}
for line in stdout_lines:
    line = line.replace(" ",";").replace(" ",";").split(";")
    line = ' '.join(line).split()
    print(line)
'''
print(stdout)

Message: reading cif file...
Message: total lines in the cif: 77
Message: determining symmetry from _symmetry_equiv_pos_as_xyz/_space_group_symop_operation_xyz ...
Message: calculated cell volume = 249.333664
CELL: name: Li1+:8.000 P1-:8.000 
CELL: vector matrix:
CELL:                5.582000    0.000000    0.000000
CELL:                0.000000    4.940000    0.000000
CELL:               -4.838119    0.000000    9.041992
CELL: parameters:    5.582000    4.940000   10.255000
CELL: angles:       90.000000  118.150000   90.000000
CELL: total atom: 16
CELL: [  0]ATOM: name=   Li1;type=Li;os= 1;x_f=(0.215,0.112,0.830);occ=1.000
CELL: [  1]ATOM: name=   Li1;type=Li;os= 1;x_f=(0.785,0.612,0.670);occ=1.000
CELL: [  2]ATOM: name=   Li1;type=Li;os= 1;x_f=(0.785,0.888,0.170);occ=1.000
CELL: [  3]ATOM: name=   Li1;type=Li;os= 1;x_f=(0.215,0.388,0.330);occ=1.000
CELL: [  4]ATOM: name=   Li2;type=Li;os= 1;x_f=(0.226,0.840,0.529);occ=1.000
CELL: [  5]ATOM: name=   Li2;type=Li;os= 1;x_f=(0.774,0.340,

In [6]:
# Testing --cal-cn-cal
cif = all_files[0]
CalCNproc = subprocess.run([cwd_exe, "--cal-cn-bv", '../test/Combine/100465.cif', 'Li1'], cwd=r'd:\Study\softBV_mix\GitHub\projects\Coord\bin', capture_output=True)
stdout = str(CalCNproc.stdout, "utf-8")
stdout_lines = stdout.split("\r\n")
for line in stdout_lines:
    line = line.split()
    print(line) 

['Message:', 'reading', 'cif', 'file...']
['Message:', 'total', 'lines', 'in', 'the', 'cif:', '77']
['Message:', 'determining', 'symmetry', 'from', '_symmetry_equiv_pos_as_xyz/_space_group_symop_operation_xyz', '...']
['Message:', 'calculated', 'cell', 'volume', '=', '249.333664']
['Message:', 'charge', 'of', 'Li(1)', 'is', 'capped', 'to', '-1.#J.']
['Message:', 'sf=0.750000']
[]


In [133]:
def fillSitesFromStdout(stdout):    
    stdout_lines = stdout.split("\r\n")
    Sites = {}
    for line in stdout_lines:
        line = line.replace(" ",";").replace("=",";").split(";")
        line = ' '.join(line).split()
        if len(line)<2: continue
        if line[1] == 'name:':
            name = ' '.join(line[2:])
        if len(line)<9: continue
        if line[3] == "name":    
            if line[4] not in Sites:
                Sites[line[4]]= {}
                Sites[line[4]]['type'] = line[6]
                Sites[line[4]]['os'] = line[8]
            else:
                continue
    return Sites, name

In [216]:
def fillSiteDic(Sites,cwd_exe,cif):
    SiteDic = {}
    for site in Sites.keys():
        Coor = [[]]
        center = ""
        CalCNproc = subprocess.run([cwd_exe, "--cal-cn-bv", cif, site], cwd=r'd:\Study\softBV_mix\GitHub\projects\Coord\bin', capture_output=True)
        stdout = str(CalCNproc.stdout, "utf-8")
        stdout_lines = stdout.split("\r\n")
        for line in stdout_lines:
            line = line.split()
            if len(line) < 1: continue
            if line[1] == "Center":
                center = line[3]
            if line[1] == "XYZ:":
                Coor[0].append(center)
                Coor[0].append((float(line[3]),float(line[4]),float(line[5])))
            if line[1] == "Coordination":
                CN = float(line[3])
            if line[1] == "BVS":
                SDofBV = float(line[6])
            if len(line) < 12: continue
            if line[5].isalpha(): continue
            if float(line[5]) <= CN:
                Coor.append([line[2]])
                Coor[-1].append((float(line[10]),float(line[11]),float(line[12])))
        if len(center) != 0: 
            SiteDic[center] = {}
            SiteDic[center]['CN'] = CN
            SiteDic[center]['SDofBV'] = SDofBV
            SiteDic[center]['Coordinates'] = Coor
            SiteDic[center]['Element'] = Sites[site]['type']
            SiteDic[center]['OS'] = Sites[site]['os']
        else: 
            print("Problematic cif:",cif,Sites,"\n")
    return SiteDic

In [101]:
def calAngleToSiteDic(SiteDic):
    for site in SiteDic:
        l = len(SiteDic[site]["Coordinates"])
        Angles = []
        for i in range(1,l-1):
            for j in range(i+1,l):
                Angles.append(calAngleDeg(SiteDic[site]["Coordinates"][0][1], SiteDic[site]["Coordinates"][i][1], SiteDic[site]["Coordinates"][j][1]))
        Angles.sort(reverse=True)
        SiteDic[site]["Angles"]= Angles
    return SiteDic

In [111]:
AngleDic = {}
for site in SiteDic:
    AngleDic[site] = []
    l = len(SiteDic[site][1])
    for i in range(1,l-1):
        for j in range(i+1,l):
            AngleDic[site].append(calAngle(SiteDic[site][1][0][1], SiteDic[site][1][i][1], SiteDic[site][1][j][1]))
    AngleDic[site].sort(reverse=True)
print(AngleDic)

{'F3': [2.602, 2.554, 2.465, 2.465, 2.431, 2.431, 2.307, 2.307, 2.276, 2.276, 1.678, 1.677, 1.619, 1.619, 1.489, 1.489, 1.472, 1.472, 1.257, 1.257, 1.249, 1.242, 1.239, 1.239, 1.211, 1.211, 1.19, 1.19], 'F1': [2.997, 2.934, 2.744, 1.79, 1.784, 1.739, 1.645, 1.633, 1.573, 1.561, 1.511, 1.496, 1.441, 1.396, 1.374], 'Cs1': [2.744, 2.723, 2.696, 2.49, 2.438, 2.422, 2.382, 2.307, 2.306, 2.199, 2.177, 2.173, 2.031, 2.005, 1.934, 1.931, 1.906, 1.903, 1.902, 1.885, 1.82, 1.814, 1.767, 1.746, 1.55, 1.535, 1.487, 1.487, 1.464, 1.434, 1.342, 1.332, 1.31, 1.309, 1.041, 1.034, 1.002, 0.981, 0.975, 0.966, 0.963, 0.955, 0.948, 0.87, 0.861], 'Li1': [2.052, 2.021, 1.91, 1.884, 1.88, 1.701], 'F2': [2.691, 2.545, 2.545, 2.405, 2.375, 2.375, 2.306, 2.306, 2.276, 2.276, 1.677, 1.677, 1.626, 1.626, 1.537, 1.537, 1.416, 1.416, 1.322, 1.322, 1.264, 1.238, 1.238, 1.228, 1.207, 1.207, 1.122, 1.121]}


In [85]:
import numpy as np

def calAngleCos(V0,V1,V2):
    V0 = np.asarray(V0)
    V1 = np.asarray(V1)
    V2 = np.asarray(V2)
    Va = V1 - V0   
    Vb = V2 - V0
    cos = np.dot(Va/np.linalg.norm(Va), Vb/np.linalg.norm(Vb))
    cos = round(cos, 4)
    cos = cos.tolist()
    return cos
# Calculate the angle radians
def calAngle(V0,V1,V2):
    V0 = np.asarray(V0)
    V1 = np.asarray(V1)
    V2 = np.asarray(V2)
    Va = V1 - V0   
    Vb = V2 - V0
    dot_product = round(np.dot(Va/np.linalg.norm(Va), Vb/np.linalg.norm(Vb)),4)
    angle = np.arccos(dot_product)
    angle = np.around(angle, 3)
    angle = angle.tolist()
    return angle

def calAngleDeg(V0,V1,V2):
    V0, V1, V2 = [np.asarray(i) for i in [V0,V1,V2]]
    Va = V1 - V0   
    Vb = V2 - V0
    dot_product = round(np.dot(Va/np.linalg.norm(Va), Vb/np.linalg.norm(Vb)),4)
    angle = np.arccos(dot_product)
    deg = np.rad2deg(angle)
    deg = np.around(deg,3)  
    deg = deg.tolist()
    return deg


In [205]:
import json
with open('../test/cifDic.json', 'w') as fp:
    json.dump(cifDic,fp)


In [164]:
# Fuction to return database_unitary dataframe
def UnitaryDF():
    skipRow =[x for x in range(0,12)]+[13,14]+[x for x in range(206,217)]
    uni_df = pd.read_fwf("database_unitary.dat",skiprows=skipRow)
    return uni_df

In [219]:
# Load cifDic into AngleList
AngleList = [[]]
i=0
for cif in cifDic:
    if len(cifDic[cif]) < 1:
        print(cif)
        continue
    for site in cifDic[cif]['Sites']:
        # Append cif name
        AngleList[i].append(cif)
        # Append chemial name
        AngleList[i].append(cifDic[cif]['Name'])
        # Append Site
        AngleList[i].append(site)
        # Append OS
        AngleList[i].append((cifDic[cif]['Sites'][site]['OS']))
        # Append Period
        AngleList[i].append((cifDic[cif]['Sites'][site]['Period']))
        # Append Group
        AngleList[i].append((cifDic[cif]['Sites'][site]['Group']))
        # Append CN
        AngleList[i].append(float(cifDic[cif]['Sites'][site]['CN']))
        # Append Angle list
        AngleList[i].append(cifDic[cif]['Sites'][site]['Angles'])
        # Add next column
        AngleList.append([])
        i += 1

In [220]:
AngleList

[['10062.cif',
  'Li1+:24.000 O2-:18.000 Pb4+:3.000',
  'Li1',
  '1',
  2,
  1,
  4.0,
  [122.425, 111.131, 110.2, 106.296, 103.993, 101.046]],
 ['10062.cif',
  'Li1+:24.000 O2-:18.000 Pb4+:3.000',
  'Li2',
  '1',
  2,
  1,
  6.0,
  [169.866,
   169.866,
   169.866,
   97.863,
   97.863,
   97.863,
   90.699,
   90.699,
   90.699,
   86.25,
   86.244,
   86.244,
   84.445,
   84.445,
   84.445]],
 ['10062.cif',
  'Li1+:24.000 O2-:18.000 Pb4+:3.000',
  'O1',
  '-2',
  2,
  6,
  7.0,
  [163.416,
   160.769,
   156.97,
   131.46,
   123.897,
   122.806,
   119.657,
   112.08,
   100.003,
   96.344,
   89.301,
   85.923,
   82.079,
   81.715,
   76.408,
   76.161,
   74.538,
   74.121,
   73.704,
   72.95,
   71.271]],
 ['10062.cif',
  'Li1+:24.000 O2-:18.000 Pb4+:3.000',
  'Pb1',
  '4',
  6,
  4,
  6.0,
  [180.0,
   180.0,
   180.0,
   91.455,
   91.455,
   91.455,
   91.455,
   91.455,
   91.455,
   88.545,
   88.545,
   88.545,
   88.545,
   88.545,
   88.545]],
 ['10257.cif',
  'Li1+:6

In [222]:
import csv

with open('../test/cifDic_1.txt', 'w',newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    for row in AngleList:
        spamwriter.writerow(row)